In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("taxi-fare-prediction")\
                            .config("spark.executor.memory", MAX_MEMORY)\
                            .config("spakr.driver.memory", MAX_MEMORY)\
                            .getOrCreate()

22/04/20 16:41:38 WARN Utils: Your hostname, devkhk-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.30.1.27 instead (on interface en0)
22/04/20 16:41:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/20 16:41:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# spark.sparkContext.getConf().getAll()

In [23]:
# 데이터 담긴 디렉토리 파일
trips_dir = "/Users/devkhk/Documents/data-engineering-study/data/trips/*"
trips_df = spark.read.csv(f"file:///{taxi_dir}", inferSchema=True, header=True)

In [24]:
trips_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [25]:
trips_df.createOrReplaceTempView("trips")

In [26]:
query = """
SELECT
    trip_distance,
    total_amount
FROM
    trips
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-08-01'

"""
comb_df = spark.sql(query)

In [30]:
comb_df.describe().show()

+-------+------------------+------------------+
|summary|     trip_distance|      total_amount|
+-------+------------------+------------------+
|  count|          13126040|          13126040|
|   mean|2.8820930920520915|17.973158757890285|
| stddev| 3.820306480671185|12.975904680786682|
|    min|              0.01|              0.01|
|    max|             475.5|            4973.3|
+-------+------------------+------------------+



In [31]:
comb_df.show()

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|         16.5|       70.07|
|         1.13|       11.16|
|         2.68|       18.59|
|         12.4|        43.8|
|          9.7|        32.3|
|          9.3|       43.67|
|         9.58|        46.1|
|         16.2|        45.3|
|         3.58|        19.3|
|         0.91|        14.8|
|         2.57|        12.8|
|          0.4|         5.3|
|         3.26|        17.3|
|        13.41|       47.25|
|         18.3|       61.42|
|         1.53|       14.16|
|          2.0|        11.8|
|         16.6|       54.96|
|         15.5|       56.25|
|          1.3|        16.8|
+-------------+------------+
only showing top 20 rows



In [39]:
# trip_distance 를 벡터로 변경하기
from pyspark.ml.feature import VectorAssembler
vassembler = VectorAssembler(inputCols=["trip_distance"], outputCol="features")

vcomb_df = vassembler.transform(comb_df)

In [40]:
vcomb_df.show()

+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         16.5|       70.07|  [16.5]|
|         1.13|       11.16|  [1.13]|
|         2.68|       18.59|  [2.68]|
|         12.4|        43.8|  [12.4]|
|          9.7|        32.3|   [9.7]|
|          9.3|       43.67|   [9.3]|
|         9.58|        46.1|  [9.58]|
|         16.2|        45.3|  [16.2]|
|         3.58|        19.3|  [3.58]|
|         0.91|        14.8|  [0.91]|
|         2.57|        12.8|  [2.57]|
|          0.4|         5.3|   [0.4]|
|         3.26|        17.3|  [3.26]|
|        13.41|       47.25| [13.41]|
|         18.3|       61.42|  [18.3]|
|         1.53|       14.16|  [1.53]|
|          2.0|        11.8|   [2.0]|
|         16.6|       54.96|  [16.6]|
|         15.5|       56.25|  [15.5]|
|          1.3|        16.8|   [1.3]|
+-------------+------------+--------+
only showing top 20 rows



In [41]:
# 학습용, 테스트용 데이터 분할
train_df, test_df = vcomb_df.randomSplit([.8, .2], seed=1)

In [42]:
from pyspark.ml.regression import LinearRegression

In [43]:
lr = LinearRegression(
        maxIter=30,
        labelCol="total_amount",
        regParam=0.01
)

In [44]:
# 모델 학습
model = lr.fit(train_df)

22/04/20 17:26:22 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/04/20 17:26:22 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/04/20 17:26:31 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [46]:
predictions = model.transform(test_df)

In [48]:
predictions.cache()

DataFrame[trip_distance: double, total_amount: double, features: vector, prediction: double]

In [47]:
predictions.show()

+-------------+------------+--------+-----------------+
|trip_distance|total_amount|features|       prediction|
+-------------+------------+--------+-----------------+
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.3|  [0.01]|9.437003859544042|
|         0.01|         3.8|  [0.01]|9.437003859

In [53]:
distance = [5.0, 15.0, 20.0, 25.5]
from pyspark.sql.types import FloatType

distance_df = spark.createDataFrame(distance, FloatType()).toDF("trip_distance")

In [57]:
vdistance_df = vassembler.transform(distance_df)

In [61]:
selected_predictions = model.transform(vdistance_df)

In [62]:
selected_predictions.show()

+-------------+--------+------------------+
|trip_distance|features|        prediction|
+-------------+--------+------------------+
|          5.0|   [5.0]| 24.26910638151479|
|         15.0|  [15.0]|53.992758730153554|
|         20.0|  [20.0]| 68.85458490447293|
|         25.5|  [25.5]| 85.20259369622426|
+-------------+--------+------------------+



In [63]:
model.summary.r2

0.7648629263876808

In [64]:
model.summary.rootMeanSquaredError

6.307820185476748

In [80]:
# 리스크를 넣으면 결과를 예측해 주는 함수로 만들어보기
def taxi_fare_predict(dl):
    dl = list(map(lambda x:float(x), dl))
    from pyspark.sql.types import FloatType
    df = spark.createDataFrame(dl, FloatType()).toDF("trip_distance")
    
    from pyspark.ml.feature import VectorAssembler
    vassembler = VectorAssembler(inputCols=["trip_distance"], outputCol="features")
    vdf = vassembler.transform(df)
    
    # 모델 불러오기 (생략)
    
    # 예측
    predictions = model.transform(vdf)
    
    return predictions.select(["trip_distance", "prediction"]).show()


In [84]:
taxi_fare_predict([1, 2, 3, 4,4.1, 22.5, 30.2, 57.9, 58])

+-------------+------------------+
|trip_distance|        prediction|
+-------------+------------------+
|          1.0|12.379645442059282|
|          2.0|15.352010676923157|
|          3.0|18.324375911787037|
|          4.0| 21.29674114665091|
|          4.1| 21.59397738667046|
|         22.5| 76.28549799163262|
|         30.2| 99.17271256781918|
|         57.9| 181.5072318412833|
|         58.0|181.80446382930026|
+-------------+------------------+



In [85]:
spark.stop()